# Lecture Objectives

- Learn how to connect to and run Postgres queries from Python
- Understand psycopg2's cursors, executes, and commits
- Learn how to generate dynamic queries through Python string formatting

##Combining SQL and Python

Often you will find yourself working with data that are only accessable through SQL.  However, your machine learning capabilities are built in Python.  To resolve this issue, we can simply set up a connection from Python to the SQL database to bring the data to us.

## Why do we care?

- SQL based databases are extremely common in almost all industry environments
- Can leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
- Very useful for scaled data pipelines, pre-cleaning, data exploration
- Allows for dynamic query generation and hence automations

## psycopg2

- A Python library that allows for connections with PostgresSQL databases to easily query and retrieve data for analysis.
- [Documentation--Includes Installation Instructions](http://initd.org/psycopg/docs/install.html)
- In addition to what's listed in the documentation, if you have the anaconda distribution of Python 
```python 
conda install psycopg2 
```
worked for me
- There are similar packages for other flavors of SQL that work much the same way

## General Workflow

1. Establish connection to Postgres database using psycopg2
2. Create a cursor
3. Use the cursor to execute SQL queries
4. Commit SQL actions
4. Close the cursor and connection

# Walkthrough 1: Creating a database from Python

### Connect to the database
- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you need to create a database, you can first connect to Postgres using the dbname 'postgres' to initialize

In [ ]:
import psycopg2 as pg2
conn = pg2.connect(dbname='postgres', user='clayton.schupp', host='localhost')

### Commits

- Data changes are not actually stored until you choose to commit
- You can choose to have automatic commit by using ` autocommit = True`
- When connecting directly to the Postgres Server to initiate server level commands such as creating a database, you must use the `autocommit = True` option since Postgres does not have "temporary" transactions at the database level

In [ ]:
conn.autocommit = True

### Instantiate the Cursor

- A cursor is a control structure that enables traversal over the records in a database
- Executes and fetches data
- When the cursor points at the resulting output of a query, it can only read each observation once.  If you choose to see a previously read observation, you must rerun the query. 
- Can be closed without closing the connection

In [ ]:
cur = conn.cursor()

###  Create a database

In [ ]:
cur.execute('DROP DATABASE IF EXISTS temp;')
cur.execute('CREATE DATABASE temp;')

### Disconnect from the cursor and database
- Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

In [ ]:
cur.close() # This is optional
conn.close() # Closing the connection also closes all cursors

# Walkthrough 2: Lets use our new database

### Connect to our database

In [ ]:
conn = pg2.connect(dbname='temp', user='clayton.schupp', host='localhost')

In [ ]:
cur = conn.cursor()

### Create a new table

In [ ]:
query = '''
        CREATE TABLE logins (
            userid integer, 
            tmstmp timestamp, 
            type varchar(10)
        );
        '''
cur.execute(query)

### Insert data into new table

In [ ]:
query = '''
        COPY logins 
        FROM '/Users/clayton.schupp/cwschupp/galvanize/teaching/DSI_Lectures/sql-python/clayton/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        '''
cur.execute(query)

### Run a query to get 30 records from our data

In [ ]:
query = '''
        SELECT *
        FROM logins
        LIMIT 30;
        '''
cur.execute(query)

### Lets look at our data one line at a time

In [ ]:
cur.fetchone()

### Many lines at a time

In [ ]:
#fetchmany(n) to get n rows
cur.fetchmany(10)

### Or everything at once

In [ ]:
#fetchall() grabs all remaining rows
cur.fetchall()

# Dynamic Queries

- A Dynamic Query is a query that generates based on context.


### Example

We have 8 login csv files that we need to insert into the logins table.  Instead of doing a COPY FROM query 8 times, we should utilize Python (or any future languages) to make this more efficient.  This is possible due to tokenized strings.

### First lets get an idea of how many records we start with

In [ ]:
cur.execute('SELECT count(*) FROM logins;')
cur.fetchall()

### os is needed because we want to dynamically identify the files we need to insert using listdir.

In [ ]:
import os

### Create a query template and determine file path for imports

### Use string formatting to generate a query for each approved file.

### [WARNING: BEWARE OF SQL INJECTION](http://initd.org/psycopg/docs/usage.html)
### NEVER use + or % to reformat strings to be used with .execute

In [ ]:
num = 579
terribly_unsafe = "SELECT * FROM logins WHERE userid = " + str(num)
print terribly_unsafe


date_cut = "2014-08-01"
horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s" % date_cut
print horribly_risky
## Python is happy, but if num or date_cut included something malicious
## your data could be at risk

In [ ]:
query = '''
        COPY logins 
        FROM %(file_path)s
        DELIMITER ','
        CSV;
        '''

folder_path = '/Users/clayton.schupp/cwschupp/galvanize/teaching/DSI_Lectures/sql-python/clayton/logins_data/'

### Use string formatting to generate a query for each approved file.

In [ ]:
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name != 'logins01.csv':
        path=folder_path+file_name
        cur.execute(query, {'file_path':path})
        print '{0} inserted into table.'.format(file_name)

### Lets check the total number of records we have right now.

In [ ]:
cur.execute('SELECT count(*) FROM logins;')
cur.fetchall()

### Don't forget to commit your changes

In [ ]:
conn.commit()

### Close your connection

In [ ]:
conn.close()

# Key Things to Remember

* Connections must be established using an existing database, username, database IP/URL, and maybe passwords
* If you have no created databases, you can connect to Postgres using the dbname 'postgres' to initialize db commands
* Data changes are not actually stored until you choose to commit. This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions is only temporary stored.
* Autocommit = True is necessary to do database commands like CREATE DATABASE.  This is because Postgres does not have temporary transactions at the database level.
* If you ever need to build similar pipelines for other forms of database, there are libraries such Pyodbc which operates essentially the same
* SQL connection databases utilizes cursors for data traversal and retrieval.  This is kind of like an iterator in Python.
* Cursor operations typically goes like the following:
    - execute a query
    - fetch rows from query result if it is a SELECT query
    - because it is iterative, previously fetched rows can only be fetched again by rerunning the query
    - close cursor through .close()
* Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

# Exercise

You're given a file called `playgolf.csv` in the data folder.  The file is comma delimited and the first row is the header.  Without opening and looking at the file, create a table and insert the data. Here is the header and first row:


|Date|Outlook|Temperature|Humidity|Windy|Result|
|----|-------|-----------|--------|-----|------|
|07-01-2014|sunny|85|85|false|Don't Play|

In [ ]:
conn = pg2.connect(dbname='temp', user='clayton.schupp', host='localhost')

In [ ]:
cur = conn.cursor()

In [ ]:
query = '''
        CREATE TABLE play_golf (
            date date,
            outlook varchar(30),
            temp integer, 
            humidity integer,
            windy boolean,
            result varchar(30)
        );
        '''
cur.execute(query)

In [ ]:
query = '''
        COPY play_golf 
        FROM '/Users/clayton.schupp/cwschupp/galvanize/teaching/DSI_Lectures/sql-python/clayton/playgolf.csv' 
        DELIMITER ',' 
        HEADER 
        CSV;
        '''
cur.execute(query)

In [ ]:
query = '''
        SELECT *
        FROM play_golf;
        '''
cur.execute(query)

In [ ]:
cur.fetchall()

In [153]:
import pandas as pd
df=pd.read_sql(query, conn)
df

,date,outlook,temp,humidity,windy,result
0,2014-07-01,sunny,85,85,False,Don't Play
1,2014-07-02,sunny,80,90,True,Don't Play
2,2014-07-03,overcast,83,78,False,Play
3,2014-07-04,rain,70,96,False,Play
4,2014-07-05,rain,68,80,False,Play
5,2014-07-06,rain,65,70,True,Don't Play
6,2014-07-07,overcast,64,65,True,Play
7,2014-07-08,sunny,72,95,False,Don't Play
8,2014-07-09,sunny,69,70,False,Play
9,2014-07-10,rain,75,80,False,Play


In [ ]:
conn.close()